# <mark> <b> > 2.0 </b> Pipeline de Extracao de dados de documentos - NLP </mark>

<b>2_extract_pipeline_NLP_V0.ipynb</b>    |     Atual notebook com as funçoes para processamento de documentos com soluçao NLP

### Modules e config

In [ ]:
import os
import sys
import shutil
import platform
import subprocess
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract


import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import spacy
from spacy import displacy
import pandas as pd
from spacy.tokens import Span
from spacy.matcher import Matcher
from spacy.tokens import Token
from spacy.language import Language
from difflib import SequenceMatcher # verificar

nlp = spacy.load("pt_core_news_sm")


import logging

# Modulos da solucao
# import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron
import modules.nova_extracao_pdf_pesquisavel as novaextra 
import modules.trata_model as tmod
import modules.trata_pdf as tpdf
import modules.utils as utl

ner = nlp.remove_pipe('ner')
ruler = nlp.add_pipe("entity_ruler")

## <mark> <b>1.0</b> Configuraçoes, funcoes e patterns NLP  </mark>

In [ ]:
# Tipo de documento em uso
tipo_doc_work = 'nfs_e'

# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'



# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"

# 4. XXX Tipos de documentos para extracao
tipo_documento_path = "config/tipo_documentos"

# 5. XXX Path para tipo de documento patterns
tipo_documento_patterns_path = "config/tipo_documentos/patterns"

# 6. XXX Nome do caminha para dict Tipo de documento
config_tipo_doc_path = "config/tipo_documentos/tipo_documento.json"

# Paths de trabalho para Raster_PDF
raster_process_pdf_path = 'processamentos/temp/pdf'
raster_process_txt_path = 'processamentos/temp/txt'


# 6. IMPORTANTE - MUDOU - Path para gestao de imagens resized
image_resized_path = "processamentos/temp/images/processadas"


#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


####Config Processamento Pipeline

# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"



# 7. path para arquivos json
json_path = "processamentos/jsons"

# 7. Path para DFs e CSVs exportados
export_path = "pipeline_extracao_documentos/6_geral_administacao/exports"

# 8. Path para lixeira
root_garbage_path = "pipeline_extracao_documentos/0_lixeira"


#### paths de objetos para criacao/gestao (dicionarios/datasets)
cnae_dict_path = "pipeline_extracao_documentos/6_geral_administacao/datasets/CNAE_X_ITEM_SERVICO_PREFEITURAS.xlsx"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

#Modelo atual
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

# definindo localizadcao para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")

# logging.basicConfig(
#     filename='config/log_ocorrencias.log',
#     level=logging.INFO, 
#     format='%(asctime)s - %(levelname)s - %(message)s', 
#     datefmt='%d/%m/%Y %H:%M:%S'
# )

# logging.info("kernel reiniciado")

### <mark> <b>1.0</b> Funcoes legadas e NLP </mark>

In [ ]:
#===================================================================================================#
#                                                                                                   #
#                                       FUNCOES LEGADAS                                             #
#                                                                                                   #
#===================================================================================================#

# XXX IMPORTANTE - ESTA E A FUNCAO PARA SER UTILIZADA: POIS CONVERTE PARA CINZA E RESIZE: (4134, 5846)
def convert_resize_gray(original_file_name, file_path, image_resized_path):

    name_image = utl.conv_filename_no_ext(original_file_name)
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    pages = convert_from_path(file_path, 500, poppler_path=poppler_path)
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((4134, 5846))
            resized_pages.append(resized_page)
            
    imagem_gray = resized_pages[0].convert('L')
    imagem_gray.save(image_resized_name, 'JPEG')

    return  imagem_gray, image_resized_name

# XXX Pequenos mas poderosos
def extract_text_PIL(image, coordinates):
    x0, y0, x1, y1 = coordinates
    image_croped = image.crop((x0, y0, x1, y1))
    texto_extraido = pytesseract.image_to_string(image_croped, lang='por', config='--psm 6')
    return texto_extraido 


# 5. XXX Ajusta textoYYY
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


##### Frame functions

# funçao importante para buscar coordenadas do frame em funçao do contexto
def get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo):
    
    row_frame = utl.filtrar_df(frames_nf_v4_df, model=model_map, context_mapping=context_mapping, type=tipo)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates



# XXXpara buscar melhor as coordendas dos FRAMES
def get_coordinates_filter(pdf_pesquisavel_map, model, tipo, label, section):
    
    row_frame = utl.filtrar_df(frames_nf_v4_df, model=model, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates




# 0.A Dados iniciais - PDF PESQUISAVEL	
def pesquisa_prefeitura_pdf_pesquisavel(idx, row, row_info, map_directory, original_file_name, file_path, debug):    
    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    if debug:
        print(f'\ndentro da funçao: pesquisa_prefeitura_pdf_pesquisavel: doc.:{original_file_name} | diretorio: {map_directory}  text: \n\n{text}\n\n')
    
    if text:
       page_number = 0
       #print(page_number)
    else:
       page_number = 1
       #print(page_number)
    
    pdf_document.close()
   
    return text




# 0. INFOMACOES INICIAIS - RASTER PDF
def processar_dados_iniciais(idx, row, row_info, section, map_directory, original_file_name, file_path, debug):
    
    # lista_texto_extraido = []

    nf_dados_doc = {}
    nf_dados_doc['secao'] = section
    pdf_pesquisavel = None
    extracted_txt = pesquisa_prefeitura_pdf_pesquisavel(idx, row, row_info, map_directory, original_file_name, file_path, debug)
    if debug:
        print(f'\n1. funcao: processar_dados_iniciais: doc.:{original_file_name} | diretorio: {map_directory} apos funcao: pesquisa_prefeitura_pdf_pesquisavel: extracted_txt:\n{extracted_txt}\n\n')
    
    if extracted_txt:
        pdf_pesquisavel = True
        print(f'extracted_txt: {extracted_txt} - portanto pdf_pesquisavel: {pdf_pesquisavel} ')
        
    else:
        pdf_pesquisavel = False
        print(f'extracted_txt: {extracted_txt} - portanto pdf_pesquisavel: {pdf_pesquisavel} ') 
       
       
        # WTF
        x0 = 220
        y0 = 0
        x1= 3858
        y1 = 1572
        
        # usando novo processo que gera o arquivo "on the fly" imagem_gray (converte PDF para imagem de tamanho grande (4134, 5846) - torna-a cinza e a salva)
        imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)
        extracted_txt = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
        #print(f'extracted_txt: {extracted_txt}')
        if debug:
            print(f'\n2. funcao: processar_dados_iniciaisdoc.:{original_file_name} | diretorio: {map_directory}  apos : extract_text_PIL: extracted_txt:\n{extracted_txt}\n\n')
    
    nf_dados_doc['file_name'] = original_file_name    
    nf_dados_doc['pdf_pesquisavel'] = pdf_pesquisavel 
    value = {}   
    texto_tratado = texto_extraido(extracted_txt)
    value = define_dados_iniciais(idx, row, row_info, texto_tratado, debug)
    if debug:
        print(f'\n3. funcao: processar_dados_iniciais doc.:{original_file_name} | diretorio: {map_directory} | apos funcao: define_dados_iniciais() value \n{value}\n\n')
    if value:
        nf_dados_doc.update(value)
   


    return nf_dados_doc



##### Extractions fucntions


# 1.B CABECALHO XXX Funcoes de extracao -cabecalho Raster
def processar_cabecalho_R_PDF(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    data_box_valores = {}
    data_box_conferencia = {}
    data_box_valores['secao'] = section
    
    batch_name_row_info = row_info.get('batch')
    #status_documento_row_info = row_info.get('status_documento')
    information_row_info = row_info.get('informations')
    action_item_row_info = row_info.get('action_item')
    
    # Busco a imagem np do documento
    image_np_row_info = row_info.get('image_np')
    
    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info
    data_box_valores['processo'] = context_mapping
    data_box_valores['conf_cod'] = 0


                     
    
    # busco coordenadas para o contexto
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"
    
    #print(f'\n2. Dentro func: section: {section} mapping_method: {mapping_method} | context_mapping: {context_mapping} | model_map: {model_map} | original_file_name: {original_file_name}\n')
   
    # 2. usando a funcao de extracao de coordenadas por contexto    
    coordinates = get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo_4_coordinates)
    x0, y0, x1, y1 = coordinates[0]
    #print(f'x0: {x0} | y0: {y0} | x1: {x1} | y1: {y1}')
    x0 = int(x0)
    y0 = int(y0)
    x1 = int(x1)
    y1 = int(y1) 
    # 3. Cropo a imagem - novo modelo
    cropped_image_np = image_np_row_info[y0:y1, x0:x1] # ajustar nos demais
    data_box_conferencia[f'box_{context_mapping}'] = cropped_image_np
    data_box_conferencia[f'coordinates_{context_mapping}'] = coordinates
    # 4. Converto para PIL
    cropped_image_pil = Image.fromarray(cropped_image_np)
    # 6. Executo OCR
    texto_extraido = pytesseract.image_to_string(cropped_image_pil, lang='por')
    # 7. Trato o texto extraido = text_splited
    text_splited = texto_extraido_cabecalho(texto_extraido)
    if debug:
        print()
        plt.imshow(cropped_image_np)
        plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
        plt.show()
        print(f'\ncoordinates {coordinates} - \ntexto_extraido:\n{text_splited}\n')
        
    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]
    
    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        try:
            section = row_frame['section_json']
            label = row_frame['label']
            reference = row_frame['reference']
            string_pesquisa = row_frame['marcador_inicio']  
            keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_box_valores[label] = texto
            if debug:
               print(f'\nidx: {index_frame:> 3} | label: {label} |  string_pesquisa:{string_pesquisa} | dentro do try do raster PDF cabecalho - texto: \n{texto}\n\n')
        except Exception as e:
            msg = (f"{e}")
            data_box_conferencia[label] = msg
    

    # Verificações após o loop
    for key, value in data_box_valores.items():
        if key == 'numero_nota_fiscal' and value is None:
            action_item_row_info = 'BREAK_PROCESS'
            information_row_info = 'Número da Nota não encontrado'
            #logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir
        
        elif key == 'codigo_verificacao' and value != None:
            codigo_verificacao_nf = value
            tam_codigo_verificacao = len(codigo_verificacao_nf)
            data_box_valores['conf_cod'] = tam_codigo_verificacao
            
        
        elif key != 'numero_nota_fiscal' and value is None:
            logging.error(f" {batch_name_row_info} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

            
      # if value is None:
        #     logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info

    
    return data_box_valores

# 1.A CABECALHO - PDF PESQUISAVEL  
def extrai_cabecalho_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    nf_data_cabecalho = {}
    lista_erros = []
    label = "1_frame_dados_nf"
    
    batch_name_row_info = row_info.get('batch')
    information_row_info = row_info.get('informations')
    action_item_row_info = row_info.get('action_item')
    
    nf_data_cabecalho['secao'] = section
    nf_data_cabecalho['action_item'] = action_item_row_info
    nf_data_cabecalho['informations'] = information_row_info
    nf_data_cabecalho['processo'] = 'mapeamento regex - PDF pesquisavel'
    
    if debug:
        print(f'\n\n2. dentro da funçao extrai_cabecalho_PDF: batch_name: {batch_name_row_info}\n\n')
    
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    y0 = y0 * f_0
    y1 = y1 * f_1
    
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    if debug:
        print(f'\n3. x0: {x0}, y0: {y0}, x1: {x1}, y1: {y1} f_0: {f_0} f_1: {f_1} | text: \n{text} \n\n')

    try:
        numero_nota_match = re.search(r'Número da Nota:\s+(\d+)', text)
        if numero_nota_match:
            numero_nf = numero_nota_match.group(1)
            nf_data_cabecalho['numero_nota_fiscal'] = numero_nf
            #nf_data_cabecalho['informations'] = 'documento com numero de nota fiscal'
            if debug:
                print(f'\nnr_nro_nf: {nr_nro_nf} - doc: {original_file_name}\n')
        else:
            msg = (f"Número da Nota não encontrado")
            nf_data_cabecalho['numero_nota_fiscal'] = None
            information_row_info = 'Número da Nota não encontrado'
            nf_data_cabecalho['informations'] = information_row_info
            action_item_row_info = 'BREAK_PROCESS'
            nf_data_cabecalho['action_item'] = action_item_row_info
    except Exception as e:
        msg = (f"doc: {original_file_name} | numero NF nao encontrado {e}")
        nf_data_cabecalho['numero_nota_fiscal'] = None
        information_row_info = 'Número da Nota não encontrado'
        nf_data_cabecalho['informations'] = information_row_info
        action_item_row_info = 'BREAK_PROCESS'
        nf_data_cabecalho['action_item'] = action_item_row_info

    # Extrair Competência
    competencia_match = re.search(r'Competência:\s+(.+)', text)
    if competencia_match:
        nf_data_cabecalho['competencia'] = competencia_match.group(1)

    # Extrair Data e Hora de Emissão
    data_emissao_match = re.search(r'Data e Hora da Emissão:\s+(.+)', text)
    if data_emissao_match:
        nf_data_cabecalho['dt_hr_emissao'] = data_emissao_match.group(1)
        
    # Extrair codigo Verificacao
    codigo_verificacao_match = re.search(r'Código Verificação:\s+(.+)', text)
    if codigo_verificacao_match:
        codigo_verificacao_nf = codigo_verificacao_match.group(1)
        nf_data_cabecalho['codigo_verificacao'] =  codigo_verificacao_nf
        tam_codigo_verificacao = len(codigo_verificacao_nf)
        nf_data_cabecalho['conf_cod'] = tam_codigo_verificacao
        
    
    
    pdf_document.close()
    
    return nf_data_cabecalho

#===================================================================================================#
#                                                                                                   #
#                                         PRIMEIRAS FUNCOES NLP                                     #
#                                                                                                   #
#===================================================================================================#


def show_ent_new(text, patterns):
    #nlp = spacy.blank("pt")
    #ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    doc = nlp(text)
    
    tokens = []
    ents = []
    
    for ent in doc.ents:
        span = doc.char_span(ent.start_char, ent.end_char, label=ent.label_)
        ents.append(span)
        
    for token in doc:
        start = token.idx
        end = start + len(token)
        tokens.append((token.text, start, end))
        
    return doc, tokens, ents



# chunk.text, chunk.start, chunk.end, chunk.root.head.lemma_, chunk.root.dep_, chunk.doc
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


# Funcoes para salvar e carregar entity ruler patterns
def write_patterns_to_file(patterns, colors, filename):
    data = {"patterns": patterns, "colors": colors}
    with open(filename, "w") as f:
        json.dump(data, f, ensure_ascii=True, indent=2)
        

# Funcao para carregar as cores e patterns do Entity ruler        
def load_patterns_and_colors(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
        patterns = data["patterns"]
        colors = data["colors"]
    return patterns, colors 


# Salva do tipo de documento em arquivo
def save_tipo_doc_to_file(dic, filename):
    with open(filename, 'w') as f:
        json.dump(dic, f) 
        
# carrega o tipo de documento do arquivo        
def load_dict_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Salva o dict doc_content em arquivo
def save_doc_content_to_file(doc_content):
    file_doc_content_path = os.path.join(map_analise_path, 'doc_content_' + batch_name + ".json")
    with open(file_doc_content_path, 'w') as f:
        json.dump(doc_content, f)
        
# Carrega o dict doc_content em arquivo
def load_doc_content_from_file():
    file_doc_content_path = os.path.join(map_analise_path, 'doc_content_' + batch_name + ".json")
    with open(file_doc_content_path, 'r') as f:
        return json.load(f) 


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()



def run_ocrmypdf(input_file, output_file):
    command = [
        'ocrmypdf',
        '--language', 'por',
        '--deskew',
        input_file,
        output_file
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode == 0:
        print(f"OCRmyPDF completed successfully. Output saved to {output_file}.")
    else:
        print(f"OCRmyPDF failed with error: {result.stderr.decode('utf-8')}")
        
        
        


# Função para definir o atributo "is_cnpj"
@Language.component("set_cnpj_attribute")
def set_cnpj_attribute(doc):
    for i, token in enumerate(doc):
        if i < len(doc) - 1:
            next_token = doc[i + 1]
            if token.shape_ == "dd.ddd.ddd/" and next_token.shape_ == "dddd-dd":
                token._.is_cnpj = True
                next_token._.is_cnpj = True
            else:
                token._.is_cnpj = False
    return doc        


# Registro do atributo 'is_cnpj'
Token.set_extension('is_cnpj', force=True, default=False)


# Função para aplicar o matcher
@Language.component("apply_cnpj_matcher")
def apply_cnpj_matcher(doc):
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        for token in span:
            token._.is_cnpj = True
    return doc


### <mark> <b>1.1</b> Carregando o dict de tipo de documento e frames_nf_v4_df </mark>

In [ ]:
# 1. XXX Buscar proximo Batch caso nao esteja rodando email
batch_name = utl.busca_proximo_batch(conf_export_plan_path)



# 0. XXX IMPORTANTE, carregar o dict de tipo de documento 
tipo_documento_dict = load_dict_from_file(config_tipo_doc_path)



matcher_pattern_path = tipo_documento_dict.get(tipo_doc_work, {}).get('matcher_pattern_path', 'valor_padrao')
entity_ruler_pattern_path = tipo_documento_dict.get(tipo_doc_work, {}).get('entity_ruler_pattern_path', 'valor_padrao')



# 2. XXX IMPORTANTE - Modelo a ser utilizado para carregar o dict doc_content
doc_content = {}
doc_content = load_doc_content_from_file()



# 3. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")


# 4. XXX Ler a planilha e cria df_documento_recebido
df_root_pipe = pd.read_excel(file_path_root_pipe)


# 5. XXX  Ajustar o indice
df_root_pipe.set_index('document_unique_id', inplace=True)


def define_dados_iniciais(idx, row, row_info, texto_tratado, debug):
    
    dados_iniciais_nf = {}
    #status_documento_row_info = row_info.get('status_documento')
    action_item_row_info = row_info.get('action_item')
    information_row_info = row_info.get('informations')
    
    dados_iniciais_nf['action_item'] = action_item_row_info
    dados_iniciais_nf['informations'] = information_row_info
    
    print(f'\nDentro da func define_dados_iniciais:  -action_item_row_info: {action_item_row_info}')
   


    prefeitura_encontrada = None
    de_para_encontrado = None

    # 7. ZZZ Dicionário para mapear Prefeitura com sua sigla
    de_para_prefeitura = {
        "PREFEITURA DA CIDADE MAGE": "PM_MAGE",
        "PREFEITURA DA CIDADE DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA\nALDEIA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE MESQUITA": "PM_MESQUITA",
        "PREFEITURA MUNICIPAL DE DE MESQUITA": "PM_MESQUITA",
        # ... adicione 
    }
    

    templates = {
        ("PM_MAGE", None): "MAGE",
        ("PM_SPA", None): "SPA",
        ("PM_MESQUITA", None): "MESQUITA",
        ("Pague agora com o seu Pix", None): "NAO_PROCESSAR",
        # ... adicione outras combinações aqui
    }

    cnpj_encontrado = None
    # Verifique cada linha do texto
    for linha in texto_tratado:
        for pref in de_para_prefeitura.keys():
            if pref in linha:
                #print(linha)
                prefeitura_encontrada = pref
                dados_iniciais_nf['prefeitura'] = prefeitura_encontrada
                if debug:
                    print(f'\n4.funcao: define_dados_iniciais(texto_tratado) - dentro do loop for de pesquisa prefeitura - prefeitura_encontrada: \n{prefeitura_encontrada}\n\n')
    # Saímos do loop, agora vamos verificar qual template usar
    if prefeitura_encontrada:
        de_para_pm = de_para_prefeitura.get(prefeitura_encontrada)
        dados_iniciais_nf['de_para_pm'] = de_para_pm
        if debug:
            print(f'\n5.funcao: define_dados_iniciais(texto_tratado) - if prefeitura_encontrada - de_para_pm \n{de_para_pm}\n\n')
        if not de_para_pm:
            de_para_pm = de_para_prefeitura.get(prefeitura_encontrada, "NAO_PROCESSAR")
            dados_iniciais_nf['de_para_pm'] = de_para_pm
            #print(de_para_pm)
    else:
        de_para_pm = "NAO_PROCESSAR"
        action_item_row_info = 'BREAK_PROCESS'
        information_row_info = 'Nao identificado dados iniciais para o documento'
        
     
        
    # Verifique cada linha do texto
    for linha in texto_tratado:
        for de_para, cnpj in templates.keys():
            if cnpj and cnpj in linha:
                cnpj_encontrado = cnpj
                dados_iniciais_nf['cnpj_encontrado'] = cnpj_encontrado
                
                
    # Saímos do loop, agora vamos verificar qual template usar
    if de_para_pm:
        template_usar = templates.get((de_para_pm, cnpj_encontrado))
        logging.info(f'usara template {template_usar} para: {cnpj_encontrado}')
        # print(template_usar)
        dados_iniciais_nf['model'] = template_usar
        if not template_usar:
            template_usar = templates.get((de_para_pm, None), "TEMPLATE_NAO_ENCONTRADO")
            dados_iniciais_nf['model'] = 'NAO_ENC.' 
            action_item_row_info = 'BREAK_PROCESS'
            information_row_info = 'model nao encontrado'
    else:
        template_usar = "TEMPLATE_NAO_ENCONTRADO"
        dados_iniciais_nf['model'] = 'NAO_ENC.'
        action_item_row_info = 'BREAK_PROCESS'
        information_row_info = 'model nao encontrado'
        
        
        
        
    
    #Confirmando se template existe em frames    
    try:        
        f_type = 'frame'
        #template_usar = 'SAO_PEDRO_SUPERMIX'
        result = filtrar_df(frames_nf_v4_df, type=f_type, de_para_pm=de_para_pm, model=template_usar)
        model = result['model'].values[0]
        if model:
            template_oficial = model
            if model == template_usar:
                dados_iniciais_nf['model'] = template_oficial
            else:    
                template_usar = "necessario cadastrar"
                dados_iniciais_nf['model'] = "CADASTRAR"
                
            dados_iniciais_nf['model'] = template_usar
        else:
            template_usar = "necessario cadastrar"
            dados_iniciais_nf['model'] = "CADASTRAR"

                
    except Exception as e:
       error_msg = (f"Erro busca do template: {e}") 
    
    dados_iniciais_nf['action_item'] = action_item_row_info 
    dados_iniciais_nf['informations'] = information_row_info         
        
    return dados_iniciais_nf  



nf_model_path = "config/modelos/frames_nf_v11.xlsx"

#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)


# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

ver = tmod.get_template_version(frames_nf_v4_df, 'MAGE')

frames_nf_v4_df.head(5)

print(f'frames_nf_v4_df: {ver}')
print()


# Unica funcao que devera ser movida para junto das demais funcoes de dicionario
def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
    
    # Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)




df_root_pipe.head(14)

### <mark> <b>1.2</b> Carregando Entity Ruler Patterns </mark>

In [ ]:
# Entity ruler patterns
colors = {
            "secretaria": "linear-gradient(90deg, #2ADB5E, #1FA346)", # Verde Degrade
            "tipo_documento": "linear-gradient(90deg, #09D6FF, #08A0D1)", #Azul medio degrade
            "nome_prefeitura": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", # Roxo claro para lilaz - degrade bem bacana
            "nome_section": "linear-gradient(90deg, #FFA9FB, #BF7FBC)", #  lilaz - Degrade
            "nome_section": "#FFEA7F", # Laranja claro
            "INSC_MUNICIPAL": "#CCA10C", # Terracota
            "CPF_CNPJ": "#CCA10C", # Terracota
            "campos": "#AB9BFC", # Roxo claro 
            "SAFRA": "#7AECEC", # Azul bem claro
            "encerrador": "#EE8AF8" # Rosa medio
        }          

patternsPrefeitura = [
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mesquita"}], "id": "PM_MESQUITA"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mage"}], "id": "PM_MAGE"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "sao"}, {"LOWER": "pedro"}, {"LOWER": "de"}, {"LOWER": "aldeia"}], "id": "PM_SPA"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "sao"}, {"LOWER": "pedro"}, {"LOWER": "da"}, {"LOWER": "aldeia"}], "id": "PM_SPA"}

                        ]


patternsSection = [     
                        {"label": "nome_section", "pattern": [{"LOWER": "número"}, {"LOWER": "da"}, {"LOWER": "nota"}, {"ORTH": ":"}], "id": "1. CABECALHO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "prestador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "2. PRESTADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "prestador"}], "id": "2. PRESTADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "tomador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "3. TOMADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "tomador"}], "id": "3. TOMADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "discriminação"}, {"LOWER": "dos"}, {"LOWER": "serviços"}], "id": "4. DESCRIMINACAO DOS SERVIÇOS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "valor"}, {"LOWER": "total"}, {"LOWER": "da"}, {"LOWER": "nota"},{"ORTH": ":"}], "id": "5. VALOR TOTAL"},
                        {"label": "nome_section", "pattern": [{"LOWER": "cnae"}], "id": "6. CNAE e Item da Lista de Serviços"},
                        {"label": "nome_section", "pattern": [{"LOWER": "valor"}, {"LOWER": "serviços"}], "id": "7. VALORES E IMPOSTOS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "dados"}, {"LOWER": "complementares"}], "id": "8. DADOS COMPLEMENTARES"},
                        {"label": "nome_section", "pattern": [{"LOWER": "outras"}, {"LOWER": "informações"}, {"IS_PUNCT": True}, {"LOWER": "criticas"}], "id": "9. OUTRAS INFORMAÇOES / CRITICAS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "observação"},{"ORTH": ":"}], "id": "10. OBSERVACOES"}

                        ]

patternsSecretarias = [{"label": "secretaria", "pattern": [{"LOWER": "secretaria"}, {"LOWER": "municipal"}, {"LOWER": "da"}, {"LOWER": "fazenda"},], "id": "SECRETARIA"}] 


patternsTipoDocumento = [
                        {"label": "tipo_documento", "pattern": [{"LOWER": "nota"}, {"LOWER": "fiscal"}, {"LOWER": "de"}, {"LOWER": "serviços"}, {"LOWER": "eletrônica"}, {"LOWER": "-"}, {"LOWER": "nfs-e"}], "id": "NFS-e"}
                        ]


patternsCampos = [
                        {"label": "campos", "pattern": [{"LOWER": "telefone"},{"ORTH": ":"}], "id": "telefone"},
                        {"label": "campos", "pattern": [{"LOWER": "nome"},{"ORTH": "/"},{"LOWER": "razão"},{"LOWER": "social"},{"ORTH": ":"}], "id": "razao_social"},
                        {"label": "campos", "pattern": [{"LOWER": "endereço"},{"ORTH": ":"}], "id": "endereco"},
                        {"label": "campos", "pattern": [{"LOWER": "e"},{"ORTH": "-"},{"LOWER": "mail"},{"ORTH": ":"}], "id": "email"},
                        {"label": "campos", "pattern": [{"LOWER": "e-mail"}], "id": "e_mail"},
                        {"label": "campos", "pattern": [{"LOWER": "inscrição"},{"LOWER": "municipal"},{"ORTH": ":"}], "id": "inscricao_municipal"},
                        {"label": "campos", "pattern": [{"LOWER": "inscrição"},{"LOWER": "estadual"},{"ORTH": ":"}], "id": "inscricao_estadual"},
                        {"label": "campos", "pattern": [{"LOWER": "rg"},{"ORTH": ":"}], "id": "rg"},
                        {"label": "campos", "pattern": [{"LOWER": "nome"},{"LOWER": "de"},{"LOWER": "fantasia"},{"ORTH": ":"}], "id": "nome_fantasia"},
                        {"label": "campos", "pattern": [{"LOWER": "cpf"},{"ORTH": "/"},{"LOWER": "cnpj"},{"ORTH": ":"}], "id": "cpf_cnpj_com_mascara"},
                        {"label": "campos", "pattern": [{"LOWER": "exigibilidade"},{"LOWER": "iss"}], "id": "exigibilidade_iss"},
                        {"label": "campos", "pattern": [{"LOWER": "regime"},{"LOWER": "tributação"}], "id": "regime_tributacao"},
                        {"label": "campos", "pattern": [{"LOWER": "simples"},{"LOWER": "nacional"}, {"IS_TITLE": "False"}], "id": "simples_nacional"},
                        {"label": "campos", "pattern": [{"LOWER": "issqn"},{"LOWER": "retido"}], "id": "issqn_retido"},
                        {"label": "campos", "pattern": [{"LOWER": "local"},{"ORTH": "."},{"LOWER": "prestação"},{"LOWER": "serviço"}], "id": "local_pretacao_servico"},
                        {"label": "campos", "pattern": [{"LOWER": "local"},{"LOWER": "incidência"}], "id": "local_incidencia"},
                        ]




# patternsValores = [{"label": "INSC_ESTADUAL", "pattern": [{"TEXT": {"REGEX": "^\d{8}$"}}], "id": "inscricao_estadual"},
#                    {"label": "INSC_MUNICIPAL", "pattern": [{"TEXT": {"REGEX": "^\d{7}$"}}], "id": "inscricao_municipal"},
#                    {"label": "CPF_CNPJ", "pattern": [{"ORTH": {"REGEX": "^\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}$"}}], "id": "cpf_cnpj_com_mascara"}
#                    ]



patternsEncerradores = [{"label": "encerrador", "pattern": [{"LOWER": "https"},{"ORTH": ":"}], "id": "encerrador"},
                        {"label": "encerrador", "pattern": [{"LOWER": "https://nfs-e.mage.rj.gov.br"}], "id": "encerrador"}
                        ] 




patternsCnpj = [
    {
        "label": "CNPJ",
        "pattern": [
            {"ORTH": {"REGEX": "^\d{2}\.\d{3}\.\d{3}/$"}},
            {"ORTH": {"REGEX": "^\d{4}-\d{2}$"}}
        ]
    }
]


patterns = patternsPrefeitura + patternsSection + patternsSecretarias + patternsTipoDocumento + patternsEncerradores + patternsCampos

In [ ]:
# # XXX Processo para salvar entity ruler pattern para json

# nome_entityruler_pattern_json = tipo_doc_work + "_entity_ruler_pattern.json"
# entityruler_file_path = os.path.join(tipo_documento_patterns_path, nome_entityruler_pattern_json)

# write_patterns_to_file(patterns=patterns, colors=colors, filename=entityruler_file_path)

In [ ]:
# # XXX Processo de load de  patterns do Entity Ruler

# entity_ruler_pattern_path = tipo_documento_dict.get(tipo_doc_work, {}).get('entity_ruler_pattern_path', 'valor_padrao')

# patterns, colors = load_patterns_and_colors(entity_ruler_pattern_path)

### <mark> <b>1.3</b> Tratando os Matchers patterns </mark>

In [ ]:
# # XXX Processo para salvar matcher pattern para json
# nome_matches_pattern_json = tipo_doc_work + "_matcher_pattern.json"
# path_matches_pattern_json = os.path.join(tipo_documento_patterns_path, nome_matches_pattern_json)
# with open(path_matches_pattern_json, "w") as f:
#     json.dump(matcher_pattern_dict, f)

In [ ]:
# # XXX Rotina para carregar e atribuir ao matcher os patterns do disco
# matcher_pattern_path = tipo_documento_dict.get(tipo_doc_work, {}).get('matcher_pattern_path', 'valor_padrao')

# matcher = Matcher(nlp.vocab)
# # XXX Carregar matcher patterns do disco
# with open(matcher_pattern_path, "r") as f:
#     loaded_patterns = json.load(f)
    
# # Adicionar ao Matcher
# for label, pattern in loaded_patterns.items():
#     matcher.add(label, [pattern])    

In [ ]:
matcher = Matcher(nlp.vocab)

# Matcher Patterns
#======================================== 1. CABECALHO
# 1. Número da Nota:
numero_nota_pattern = [
    {"LOWER": "número"},
    {"LOWER": "da"},
    {"LOWER": "nota"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True}
]
matcher.add("numero_nota_fiscal", [numero_nota_pattern])


# 2. Competência:
competencia_pattern = [
    {"LOWER": "competência"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "?"},
    {"ORTH": {"REGEX": "^[A-Z][a-z]+/[0-9]{4}$"}}   
]    
matcher.add("competencia", [competencia_pattern])

# 3. Data e Hora de Emissão:
data_hora_emissao_pattern = [
    {"LOWER": "data"},
    {"LOWER": "e"},
    {"LOWER": "hora"},
    {"LOWER": "da"},
    {"LOWER": "emissão"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"SHAPE": "dd/dd/dddd"},
    {"IS_SPACE": True, "OP": "*"},
    {"SHAPE": "dd:dd:dd"}
]
matcher.add("dt_hr_emissao", [data_hora_emissao_pattern])

# 4. Código de Verificação:
codigo_verificacao_pattern = [
    {"LOWER": "código"},
    {"LOWER": "verificação"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_ASCII": True, "LENGTH": 9}
]
matcher.add("codigo_verificacao", [codigo_verificacao_pattern])


#========================================  5. VALOR TOTAL
valor_total_nota_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "total"},
    {"LOWER": "da", "OP": "?"},
    {"LOWER": "nota", "OP": "?"},
    {"TEXT": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_total_nota", [valor_total_nota_pattern])





#======================================== 7. VALORES E IMPOSTOS

# 1. VALOR_SERVICOS
valor_servicos_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "serviços"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_servicos", [valor_servicos_pattern])



# 2. VALOR DEDUÇÃO:
valor_deducao_1_pattern = [
    {"LOWER": "dedução"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_deducao", [valor_deducao_1_pattern])

# 2.1 VALOR DEDUÇÃO:
valor_deducao_2_pattern = [
    {"LOWER": "dedução"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "Number=Sing", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_deducao", [valor_deducao_2_pattern])



# 2.2 DESC. INCOND1:
valor_deducao_3_pattern= [
    {"LOWER": "dedução"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"}  
]
matcher.add("valor_deducao", [valor_deducao_3_pattern])



# 3. DESC. INCOND: - PDF_Pesquisavel   #DESC. INCOND:
desc_incond_pattern = [
    {"LOWER": "desc"},
    {"LOWER": "incond"},
    {"ORTH": ":"},
    {"IS_SPACE": True},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}  
]
matcher.add("desc_incond", [desc_incond_pattern])

# 3.1 DESC. INCOND1:
desc_incond_1_pattern = [
    {"LOWER": "base"},
    {"LOWER": "de"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"}  
]
matcher.add("desc_incond", [desc_incond_1_pattern])


# 3.2 DESC. INCOND: - PDF_Pesquisavel   #DESC. INCOND:
desc_incond_2_pattern = [
    {"LOWER": "incond"},
    {"ORTH": ":"},
    {"LOWER": "base"},
    {"LOWER": "de"},
    {"IS_SPACE": True},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}  
]
matcher.add("desc_incond", [desc_incond_2_pattern])





# 4.1 BASE DE CÁLCULO:  PDF_P
base_calculo_1_pattern = [
    {"LOWER": "base"},
    {"LOWER": "de"},
    {"LOWER": "cálculo"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("base_calculo", [base_calculo_1_pattern])


# 4.2 BASE DE CÁLCULO:  RASTER_PDF
base_calculo_2_pattern = [
    {"LOWER": "calculo"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("base_calculo", [base_calculo_2_pattern])


# 5. Aliquota
aliquota_pattern = [
            {"LOWER": "alíquota"},
            {"ORTH": ":"},
            {"SHAPE": "d", "OP": "?"},
            {"ORTH": "%"}
        ]
matcher.add("aliquota", [aliquota_pattern])

aliquota_1_pattern =  [
            {"LOWER": "alíquota"},
            {"ORTH": ":"},
            {"SHAPE": "d,dd", "OP": "?"},
            {"ORTH": "%"}
        ]
matcher.add("aliquota", [aliquota_1_pattern])


aliquota_2_pattern = [
            {"LOWER": "valor"},
            {"LOWER": "iss"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True, "OP": "+"},
            {"ORTH": "", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"},
            {"ORTH": "%"}
        ]
matcher.add("aliquota", [aliquota_2_pattern])



# 6. VALOR ISS:
valor_iss_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_iss", [valor_iss_pattern])


# 7. VALOR ISS RETIDO:
valor_iss_retido_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"LOWER": "retido"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_iss_retido", [valor_iss_retido_pattern])


# 8. DESC. COND:
desc_cond_pattern = [
    {"LOWER": "desc"},
    {"ORTH": "."},
    {"LOWER": "cond"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("desc_cond", [desc_cond_pattern])


# 9. VALOR PIS:
valor_pis_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "pis"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_pis", [valor_pis_pattern])


# 10. VALOR COFINS:
valor_cofins_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "cofins"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_cofins", [valor_cofins_pattern])


# 11. VALOR IR:
valor_ir_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "ir"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_ir", [valor_ir_pattern])



# 12. VALOR INSS:
valor_inss_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "inss"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_inss", [valor_inss_pattern])



# 13. VALOR CSLL:
valor_csll_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "csll"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_csll", [valor_csll_pattern])



# 14. OUTRAS RETENÇÕES:
outras_retencoes_pattern = [
    {"LOWER": "outras"},
    {"LOWER": "retenções"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("outras_retencoes", [outras_retencoes_pattern])


# 15. VALOR LÍQUIDO:
valor_liquido_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "líquido"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("valor_liquido", [valor_liquido_pattern])




In [ ]:
# XXX Criar um dicionário para matchers patterns - nfs-e
# Criar nova rotina de salvamento de dicionário de matchers patterns


matcher_pattern_dict = [
                            {
                                "label": "endereco_site",
                                "pattern": [
                                        {"SHAPE": "xxxx://xxx.xxxx.xx.xxx.xx"}
                                ]         
                            },
                            {
                                "label": "OUTRO_LABEL",
                                "pattern": [
                                        {"LOWER": "valor"},
                                        {"LOWER": "líquido"},
                                        {"ORTH": ":"},
                                        {"SHAPE": "X$"},
                                        {"MORPH": "NumType=Card", "OP": "+"},
                                        {"LOWER": ".", "OP": "?"},
                                        {"IS_DIGIT": True, "OP": "*"}
                                ]
                            }
                        
                        ]

# Salvando com indentações
with open("dados_formatados.json", "w") as f:
    json.dump(matcher_pattern_dict, f, indent=4)
    
    
    
# Carregar o JSON dos padrões
with open("dados_formatados.json", "r") as f:
    loaded_patterns = json.load(f)

# Adicionar os padrões ao Matcher
for item in loaded_patterns:
    label = item["label"]
    pattern = item["pattern"]
    matcher.add(label, [pattern])
    

### <mark> <b>1.4</b> Testes de templates e  patterns  </mark>

In [ ]:
# XXX Carregagando o sample documento do tipo de documento em execucao
sample_text = tipo_documento_dict.get(tipo_doc_work, {}).get('sample_content', 'valor_padrao')
sample_text

<mark> <b>1.4.1</b> Entity Ruler Patterns  </mark>

In [ ]:
# XXX Aplicar a funçao show_ent_new para exibir o resultado
doc, tokens, ents = show_ent_new(sample_text, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

<mark> <b>1.4.2</b> Matcher Patterns  </mark>

In [ ]:
doc = nlp(sample_text)

matches = matcher(doc)

spans = []
for match_id, start, end in matches:
    span = doc[start:end]
    span.label_ = nlp.vocab.strings[match_id]  # Adicionar o label ao span
    spans.append(span)
    
# Configuração de cores
colors = {
            "numero_nota_fiscal": "orange",
            "competencia": "lightblue",
            "dt_hr_emissao": "lightblue",
            "codigo_verificacao": "silver",
            "valor_total_nota": "#EE8AF8",  
            "ALIQUOTA2": "turquoise",
            "VALOR_TOTAL":"linear-gradient(90deg, #2ADB5E, #1FA346)",
            "VALOR_SERVICOS":"linear-gradient(90deg, #09D6FF, #08A0D1)",
            "VALOR_DEDUCAO": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
            "VALOR_DEDUCAO2": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
            "VALOR_CSLL": "linear-gradient(90deg, #FFA9FB, #BF7FBC)",
            "VALOR_CSLL2": "linear-gradient(90deg, #FFA9FB, #BF7FBC)",
            "VALOR_INSS": "lightblue",
            "VALOR_INSS2": "lightblue",
            "BASE_CALCULO": "lightblue",
            "VALOR_CSLL": "#FFEA7F", # Laranja claro
            "INSC_MUNICIPAL": "#CCA10C", # Terracota
            "CPF_CNPJ": "#CCA10C", # Terracota
            "campos": "#AB9BFC", # Roxo claro 
            "OUTRAS_RETENCOES": "#7AECEC", # Azul bem claro
            "VALOR_LIQUIDO": "#EE8AF8", # Rosa medio
            "VALOR_CSLL": "silver",
            "VALOR_IR": "orange",
            "VALOR_COFINS": "lime",
            "VALOR_PIS":"cyan",
            "DESC_COND": "#AB9BFC",
            "VALOR_ISS_RETIDO": "mint",
            "VALOR_ISS": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
            "BASE_DE_CALCULO": "salmon",
            "DESC_INCOND": "#AB9BFC",
            "DESC_INCOND1": "pink",
            "VALOR_DEDUCAO": "white",
            "endereco_site": "orange",
          }

# Renderizar
displacy.render([{'text': doc.text, 'ents': [{'start': span.start_char, 'end': span.end_char, 'label': span.label_} for span in spans]}], style='ent', manual=True, options={"colors": colors})    

In [ ]:
# Relaçao de valores encontrados - Matcher Patterns
matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f"{string_id:>30} | {span.text:>50} | {span.start_char:>5}  {span.end_char:>5}")

In [ ]:
amostra = "https://nfe.mesquita.rj.gov.br"
doc1 = nlp(amostra)


# Analisys
syntatic = pd.DataFrame(data=[], \
  columns=["id", "T_texto","T_shape", "T_is_alpha", "T_is_digit", "T_is_title", "T_is_punct", "T_is_sent_start", "T_is_right_punct", "T_is_stop", "T_is_quote", "T_is_currency", "T_morph"])
i = 0
for token in doc1:
    syntatic.loc[i,"id"] = token.i
    syntatic.loc[i,"T_texto"] = token.text
    syntatic.loc[i,"T_shape"] = token.shape_
    syntatic.loc[i,"T_is_alpha"] = token.is_alpha
    syntatic.loc[i,"T_is_digit"] = token.is_digit
    syntatic.loc[i,"T_is_title"] = token.is_title
    syntatic.loc[i,"T_is_punct"] = token.is_punct
    syntatic.loc[i,"T_is_sent_start"] = token.is_sent_start
    syntatic.loc[i,"T_is_right_punct"] = token.is_right_punct
    syntatic.loc[i,"T_is_stop"] = token.is_stop
    syntatic.loc[i,"T_is_quote"] = token.is_quote
    syntatic.loc[i,"T_is_currency"] = token.is_currency
    syntatic.loc[i,"T_morph"] = token.morph
    i = i+1

syntatic.head(80)

In [ ]:
# Lemmatization for tokens 
lemmatization = pd.DataFrame(data=[], \
  columns=["id", "Texto","Lemma", "Tag", "Tag_explainned", "token_POS", "POS_explainned", "dep", "T. Head", "dep explained"])
i = 0
for token in doc1:
    lemmatization.loc[i,"id"] = token.i
    lemmatization.loc[i,"Texto"] = token.text
    lemmatization.loc[i,"Lemma"] = token.lemma_
    lemmatization.loc[i,"Tag"] = token.tag_
    lemmatization.loc[i,"Tag_explainned"] = spacy.explain(token.tag_)
    lemmatization.loc[i,"token_POS"] = token.pos_
    lemmatization.loc[i,"POS_explainned"] = spacy.explain(token.pos_)
    lemmatization.loc[i,"dep"] = token.dep_
    lemmatization.loc[i,"T. Head"] = token.head.text
    lemmatization.loc[i,"dep explained"] = token.morph
    
    i = i+1
lemmatization

In [ ]:
displacy.render(doc1, style="dep", options={"colors": colors})

<mark> <b>1.3</b> Criando novos patterns </mark>

In [ ]:
nlp.add_pipe("set_cnpj_attribute") # Adicione esta etapa se você quiser definir o atributo manualmente

nlp.add_pipe("apply_cnpj_matcher")  # Adicione esta etapa para aplicar o matcher

## <mark> <b>2.0</b> Mapeamento e Extracao  </mark>

### <mark> <b>2.1</b> Funcoes mapeamento e extracao NLP  </mark>

In [ ]:
# Funcao que ajusta muito bem formataçao de numeros - moeda e %
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)



# 0.B XXX Funçao de criacao do PDF Pesquisavel
def run_ocrmypdf(input_file, output_file):
    command = [
        'ocrmypdf',
        '--language', 'por',
        '--deskew',
        '--force-ocr',
        input_file,
        output_file
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode == 0:
        print(f"OCRmyPDF completed successfully. Output saved to {output_file}.")
    else:
        print(f"OCRmyPDF failed with error: {result.stderr.decode('utf-8')}")
 
 
 
 
        
        
# 1.A XXX Extracao de texto de todo o documento - PDF PESQUISAVEL	
def extracao_texto_PDF_P(i, document_unique_id, nome_arquivo, file_path, debug):    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Extrair texto dentro do retângulo
    text_P = page.get_text("text")
    
    pdf_document.close()
    
    texto_PDF_P = re.sub('\s+', ' ', text_P).strip()  
    if debug:
        print(f'\nFUNC analise_texto_PDF: doc.:{nome_arquivo}   texto_PDF_P: \n\n{texto_PDF}\n\n')

    return texto_PDF_P  





# 1.B XXX Extracao de texto de todo o documento - RASTER PDF
def extracao_texto_Raster_P(i, document_unique_id, nome_arquivo, file_path, debug):    
    
    output_file = None
    txt_document_file = None
    
    input_file = file_path
    
    #output_file = "/home/dani-boy/extractNF/processamentos/temp/documento.pdf"
    output_file = os.path.join(raster_process_pdf_path, document_unique_id + '.pdf')

    # 1. XXX Executar o comando OCRmyPDF
    run_ocrmypdf(input_file, output_file)

    txt_document_file = os.path.join(raster_process_txt_path, document_unique_id + '.txt')

    txt_dir = os.path.dirname(txt_document_file)
    if not os.path.exists(txt_dir):
        os.makedirs(txt_dir)

    # Execute o comando
    subprocess.run(["pdftotext", output_file, txt_document_file])

    # 3. XXX Ler o arquivo TXT
    with open(txt_document_file, 'r', encoding='utf-8') as arquivo:
        texto_OCR_R = arquivo.read()
    texto_Raster_P = re.sub('\s+', ' ', texto_OCR_R).strip()
    
    os.remove(output_file)
    os.remove(txt_document_file)
    
    return texto_Raster_P  





# 2. XXX Analise de silimaridade doc1 x doc2
def analisa_similiaridade_doc(doc1, doc2):
    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([doc1, doc2])

    # Calculando Similaridade de Cosseno
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

    print(f"Score de Similaridade: {similarity_scores[0][0]}")
    
    return similarity_scores[0][0]    





# 0.A Extracao de texto de todo o documento - PDF PESQUISAVEL	
def extrai_texto_PDF_P(idx, row, row_info, section, map_directory, original_file_name, file_path, debug):    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Extrair texto dentro do retângulo
    text_P = page.get_text("text")
    
    pdf_document.close()
    
    texto_PDF_P = text_P.replace('\n', ' ') 
    if debug:
        print(f'\nFUNC extrai_texto_PDF_P: doc.:{original_file_name} | diretorio: {map_directory}  texto_PDF_P: \n\n{texto_PDF_P}\n\n')

    return texto_PDF_P







def processar_cabecalho_PDF_P(idx, row, row_info, section, matches, doc, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    data_box_valores = {}
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"
    
    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]

    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        section = row_frame['section_json']
        label = row_frame['label']
        reference = row_frame['reference']
        string_pesquisa = row_frame['marcador_inicio']  
        
        raw_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == label), None)
        
        most_similar_reference = max([reference], key=lambda x: similar(x, raw_value))
        ##print(f'\nmost_similar_reference: {most_similar_reference}\n')
        final_value = raw_value.split(":", 1)[-1].strip()
        data_box_valores[label] = final_value

    return data_box_valores 







def extrac_cabecalho_R_PDF(idx, row, row_info, doc_content, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug):   
     
    data_box_valores = {}
    data_box_valores['secao'] = section

    # Busco a imagem np do documento
    # image_np_row_info = row_info.get('image_np')
    image_np_row_info = doc_content.get(idx, {}).get('image_np', 'valor_padrao')

    # busco coordenadas para o contexto
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"

    # 2. usando a funcao de extracao de coordenadas por contexto    
    coordinates = get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo_4_coordinates)
    x0, y0, x1, y1 = coordinates[0]
    x0 = int(x0)
    y0 = int(y0)
    x1 = int(x1)
    y1 = int(y1) 
    # 3. Cropo a imagem - novo modelo
    cropped_image_np = image_np_row_info[y0:y1, x0:x1] # ajustar nos demais
    # 4. Converto para PIL
    cropped_image_pil = Image.fromarray(cropped_image_np)
    # 6. Executo OCR
    texto_extraido = pytesseract.image_to_string(cropped_image_pil, lang='por')
    # 7. Trato o texto extraido = text_splited
    texto_cabechalho_PDF_Raster = re.sub('\s+', ' ', texto_extraido).strip() 

    if debug:
        plt.imshow(cropped_image_np)
        plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
        plt.show()
        print(f'texto_cabechalho_PDF_Raster: {texto_cabechalho_PDF_Raster}\n')  
    
    return texto_cabechalho_PDF_Raster 






def ajusta_texto_Raster_P(idx, row, document_unique_id, original_text, texto_cabechalho_PDF_Raster):

    # Passa pelo mapeamento de doc ents para ajustar o texto
    doc, tokens, ents = show_ent_new(original_text, patterns=patterns)

    nome_prefeitura_start_char = [ent.start_char for ent in doc.ents if ent.label_ == 'nome_prefeitura'][0]
    nome_prefeitura_end_char = [ent.end_char for ent in doc.ents if ent.label_ == 'nome_prefeitura'][0]
    secretaria_start_char = [ent.start_char for ent in doc.ents if ent.label_ == 'secretaria'][0]
    secretaria_end_char = [ent.end_char for ent in doc.ents if ent.label_ == 'secretaria'][0]
    tipo_documento_start_char = [ent.start_char for ent in doc.ents if ent.label_ == 'tipo_documento'][0]
    tipo_documento_end_char = [ent.end_char for ent in doc.ents if ent.label_ == 'tipo_documento'][0]
    prestador_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '2. PRESTADOR DE SERVIÇO'][0]

    bloco_prefeitura = original_text[nome_prefeitura_start_char:nome_prefeitura_end_char]
    bloco_secretaria = original_text[secretaria_start_char:secretaria_end_char]
    bloco_tipo_documento = original_text[tipo_documento_start_char:tipo_documento_end_char]
    bloco_inicio_prestador = original_text[prestador_start_char:]

    recomposed_text = bloco_prefeitura + ' | ' + bloco_secretaria + ' | ' + bloco_tipo_documento  + ' | ' + texto_cabechalho_PDF_Raster + ' | ' + bloco_inicio_prestador
    
    return recomposed_text







def mapeia_cabecalho(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):

    data_box_valores = {}
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"

    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]

    # 9. iter sobre o filtro
    i = 1
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        section = row_frame['section_json']
        label = row_frame['label']
        reference = row_frame['reference']
        string_pesquisa = row_frame['marcador_inicio'] 
        # if debug:
        #print(f'\n1. label: {label} | reference: {reference} | string_pesquisa{string_pesquisa}\n') 
        
        raw_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == label), None)
        #print(f'\n2. reference: {reference} | raw_value: {raw_value}\n')
        
        most_similar_reference = max([reference], key=lambda x: similar(x, raw_value))
        #print(f'\n3. most_similar_reference: {most_similar_reference} \n')
        
        if debug:
            print(f'\nmost_similar_reference: {most_similar_reference}\n')
        final_value = raw_value.split(":", 1)[-1].strip()
        data_box_valores[label] = final_value
        #print(f'i: {i} | file_name: {original_file_name}  | label: {label} | reference: {reference} | raw_value: {raw_value} | most_similar_reference: {most_similar_reference}  \n\n')
        i += 1
    
    return data_box_valores



def mapeia_prestador(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):

    data_box_valores = {}
    
    texto = doc_content.get(idx, {}).get('content', 'valor_padrao')

    # 2. PRESTADOR DE SERVIÇO
    prestador_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '2. PRESTADOR DE SERVIÇO'][0]
    tomador_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '3. TOMADOR DE SERVIÇO'][0]
    text = texto[prestador_end_char:tomador_start_char]

    # 1. CNPJ com e sem mascara- Prestador
    match = re.search(r'CPF/CNPJ:\s+(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
    if match:
        p_cpf_cnpj_com_mascara = match.group(1).strip()
        data_box_valores['p_cpf_cnpj_com_mascara'] = p_cpf_cnpj_com_mascara
        #print(f"p_cpf_cnpj_com_mascara: {p_cpf_cnpj_com_mascara}")
        p_cpf_cnpj_sem_mascara = re.sub(r'\D', '', match.group(1))
        data_box_valores['p_cpf_cnpj_sem_mascara'] = p_cpf_cnpj_sem_mascara
        #print(f"p_cpf_cnpj_sem_mascara: {p_cpf_cnpj_sem_mascara}")
        
    # 2. Telefone Prestador
    match = re.search(r"Telefone:(.*?)Inscrição", text)
    if match:
        p_telefone = match.group(1).strip()
        data_box_valores['p_telefone'] = p_telefone
        #print(f"p_telefone: {p_telefone}")
        
    # doc = nlp(p_telefone)  

    # 3. Inscrição Municipal - Prestador
    match = re.search(r"Inscrição Municipal:(.*?)Telefone:", text)
    if match:
        p_inscricao_municipal = match.group(1).strip()
        data_box_valores['p_inscricao_municipal'] = p_inscricao_municipal
        #print(f"p_inscricao_municipal: {p_inscricao_municipal}")
        
    # doc = nlp(p_inscricao_municipal)    


    # 4. Para o campo Inscrição Estadual PRESTADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r"Inscrição Estadual:(.*?)Nome/Razão Social:", text)
    if match:
        p_inscricao_estadual = match.group(1).strip()
        data_box_valores['p_inscricao_estadual'] = p_inscricao_estadual 
    else:
        p_inscricao_estadual = None
        data_box_valores['p_inscricao_estadual'] = p_inscricao_estadual 
        
    #print(f"p_inscricao_estadual: {p_inscricao_estadual}")   

    # 5. Razao Social Prestador
    match = re.search(r"Nome/Razão Social:(.*?)Nome de Fantasia:", text)
    if match:
        razao_social_prestador = match.group(1).strip()
        data_box_valores['razao_social_prestador'] = razao_social_prestador
        #print(f"razao_social: {razao_social_prestador}")
        

    # 6. Nome de Fantasia - Prestador
    match = re.search(r"Nome de Fantasia:(.*?)Endereço:", text)
    if match:
        p_nome_fantasia = match.group(1).strip()
        data_box_valores['p_nome_fantasia'] = p_nome_fantasia
        #print(f"p_nome_fantasia: {p_nome_fantasia}")
        

    # 7. Endereço Prestador
    match = re.search(r"Endereço:(.*?)E-mail:", text)
    if match:
        endereco_prestador = match.group(1).strip()
        data_box_valores['endereco_prestador'] = endereco_prestador 
        #print(f"endereco_prestador: {endereco_prestador}")
        

    # 8. email: Prestador  - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r'E-mail:\s+(.+)', text)
    if match:
        p_email = match.group(1).strip()
        data_box_valores['p_email'] = p_email
    else:
        p_email = None
        data_box_valores['p_email'] = p_email
        

    return data_box_valores 



def mapeia_tomador(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):

    data_box_valores = {}
    
    texto = doc_content.get(idx, {}).get('content', 'valor_padrao')
    
    # 3. TOMADOR DE SERVIÇO
    tomador_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '3. TOMADOR DE SERVIÇO'][0]
    servicos_star_char = [ent.start_char for ent in doc.ents if ent.id_ == '4. DESCRIMINACAO DOS SERVIÇOS'][0]
    text = texto[tomador_end_char:servicos_star_char]

    # 1. CNPJ Tomador
    match = re.search(r'CPF/CNPJ:\s+(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
    if match:
        t_cpf_cnpj_com_mascara = match.group(1).strip()
        data_box_valores['t_cpf_cnpj_com_mascara'] = t_cpf_cnpj_com_mascara
        #print(f"t_cpf_cnpj_com_mascara: {t_cpf_cnpj_com_mascara}")
        t_cpf_cnpj_sem_mascara = re.sub(r'\D', '', match.group(1))
        data_box_valores['t_cpf_cnpj_sem_mascara'] = t_cpf_cnpj_sem_mascara
        #print(f"t_cpf_cnpj_sem_mascara: {t_cpf_cnpj_sem_mascara}")
    
    # 2. Inscrição Municipal TOMADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r"INSC:MUNICIPAL:(.*?)RG:", text)
    if match:
        t_inscricao_municipal = match.group(1).strip()
        data_box_valores['t_inscricao_municipal'] = t_inscricao_municipal
    else:
        t_inscricao_municipal = None
        data_box_valores['t_inscricao_municipal'] = t_inscricao_municipal


    #2.1 Inscrição Municipal TOMADOR - variacao de texto
    match = re.search(r"Inscrição Municipal:(.*?)RG:", text)
    if match:
        t_inscricao_municipal = match.group(1).strip()
        data_box_valores['t_inscricao_municipal'] = t_inscricao_municipal
    else:
        t_inscricao_municipal = None
        data_box_valores['t_inscricao_municipal'] = t_inscricao_municipal


    # 3.  Telefone TOMADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r"Telefone:(.*?)Inscrição Estadual:", text)
    if match:
        t_telefone = match.group(1).strip()
        data_box_valores['t_telefone'] = t_telefone
    else:
        t_telefone = None
        data_box_valores['t_telefone'] = t_telefone
        
    # 4. RG TOMADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r"RG:(.*?)Telefone:", text)
    if match:
        t_RG = match.group(1).strip()
        data_box_valores['t_RG'] = t_RG
    else:
        t_RG = None
        data_box_valores['t_RG'] = t_RG

    # 5. Inscrição Estadual TOMADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r"Inscrição Estadual:(.*?)Nome/Razão Social:", text)
    if match:
        t_inscricao_estadual = match.group(1).strip()
        data_box_valores['t_inscricao_estadual'] = t_inscricao_estadual
    else:
        t_inscricao_estadual = None
        data_box_valores['t_inscricao_estadual'] = t_inscricao_estadual

    # 6. Nome/Razão Social TOMADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r"Nome/Razão Social:(.*?)Endereço:", text)
    if match:
        t_nome_razao_social = match.group(1).strip()
        data_box_valores['t_nome_razao_social'] = t_nome_razao_social
    else:
        t_nome_razao_social = None
        data_box_valores['t_nome_razao_social'] = t_nome_razao_social
    
    # 7. Endereço: TOMADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r"Endereço:(.*?)E-mail:", text)
    if match:
        t_endereco = match.group(1).strip()
        data_box_valores['t_endereco'] = t_endereco
    else:
        t_endereco = None
        data_box_valores['t_endereco'] = t_endereco
        
    # 8. email: TOMADOR - Neste caso ele acho a correspondencia mas nao extraiu o valor pois nao ha valor
    match = re.search(r'E-mail:\s+(.+)', text)
    if match:
        t_email = match.group(1).strip()
        data_box_valores['t_email'] = t_email
    else:
        t_email = None
        data_box_valores['t_email'] = t_email
        

    return data_box_valores



def mapeia_campos_valores(idx, row, row_info, doc_content, doc, matches, section, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    # for match_id, start, end in matches:
    #     string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    #     span = doc[start:end]  # Obter o trecho correspondente
    #     #print(f"{string_id}: {span.text}")
        
    data_box_valores = {}
    context_mapping = 'context_matcher'
    tipo_4_filter = "field_box"
    #text = doc_content.get(idx, {}).get('content', 'valor_padrao')
    #print(texto)


    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]

    # 9. iter sobre o filtro
    i = 1
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        section = row_frame['section_json']
        label = row_frame['label']
        reference = row_frame['reference']
        string_pesquisa = row_frame['marcador_inicio'] 
        # if debug:
        #print(f'\n1. label: {label} | reference: {reference}\n') 
        
        raw_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == label), None)
        #print(f'\n2. reference: {reference} | raw_value: {raw_value}\n')
        try:
            most_similar_reference = max([reference], key=lambda x: similar(x, raw_value))
            
            final_value = raw_value.split(":", 1)[-1].strip()
            final_value = format_number2(final_value)
            data_box_valores[label] = final_value
            #print(f'i: {i} | file_name: {original_file_name}  | label: {label} | reference: {reference} | raw_value: {raw_value} | most_similar_reference: {most_similar_reference}  final_value: {final_value}\n')
        except Exception as e:
            #print(f'Erro: reference: {reference} raw_value: {raw_value} | {e}')
            most_similar_reference = None    
        #print(f'\n3. most_similar_reference: {most_similar_reference} \n')
        

        i += 1
    
    return data_box_valores


def mapeia_cnae_item(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):

    data_box_valores = {}
    
    texto = doc_content.get(idx, {}).get('content', 'valor_padrao')

    # 6. CNAE e Item da Lista de Serviços
    valor_cnae_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '6. CNAE e Item da Lista de Serviços'][0]
    valores_impostos_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '7. VALORES E IMPOSTOS'][0]
    text = texto[valor_cnae_start_char:valores_impostos_start_char]

    match = re.search(r"CNAE - (.*?)Item da Lista de Serviços", text)
    if match:
        cnae_value = match.group(1).strip()
        data_box_valores['cnae'] = cnae_value
        #print(f"cnae_value:{cnae_value}")
        #data_box_valores['t_telefone'] = t_telefone
    else:
        cnae_value = None
        data_box_valores['cnae'] = cnae_value
        #print(f"cnae_value: {cnae_value}")
        #data_box_valores['t_telefone'] = t_telefone
        
    match = re.search(r'Item da Lista de Serviços -\s+(.+)', text)
    if match:
        item_value = match.group(1).strip()
        data_box_valores['item_lista_servicos'] = item_value
        #print(f"item_value:{item_value}")
        #data_box_valores['t_telefone'] = t_telefone
    else:
        item_value = None
        data_box_valores['item_lista_servicos'] = item_value
        #print(f"item_value: {item_value}")
        #data_box_valores['t_telefone'] = t_telefone  
        
    return data_box_valores



def mapeia_informacoes_criticas(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):

    data_box_valores = {}
    
    texto = doc_content.get(idx, {}).get('content', 'valor_padrao')
    
    # 9. OUTRAS INFORMAÇOES / CRITICAS
    outras_informacoes_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '9. OUTRAS INFORMAÇOES / CRITICAS'][0]
    observacoes_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '10. OBSERVACOES'][0]
    text = texto[outras_informacoes_end_char:observacoes_start_char]


    match = re.search(r" EXIGIBILIDADE ISS(.*?) REGIME TRIBUTAÇÃO", text)
    if match:
        exigibilidade_iss_value = match.group(1).strip()
        data_box_valores['exigibilidade_iss'] = exigibilidade_iss_value

    else:
        exigibilidade_iss_value = None
        data_box_valores['exigibilidade_iss'] = exigibilidade_iss_value
        
    match = re.search(r"REGIME TRIBUTAÇÃO(.*?) SIMPLES NACIONAL", text)
    if match:
        regime_tributacao_value = match.group(1).strip()
        data_box_valores['regime_tributacao'] = regime_tributacao_value

    else:
        regime_tributacao_value = None
        data_box_valores['regime_tributacao'] = regime_tributacao_value

        
    match = re.search(r"SIMPLES NACIONAL(.*?) ISSQN RETIDO", text)
    if match:
        simples_nacional_value = match.group(1).strip()
        data_box_valores['simples_nacional'] = simples_nacional_value

    else:
        simples_nacional_value = None
        data_box_valores['simples_nacional'] = simples_nacional_value
        
    match = re.search(r"ISSQN RETIDO(.*?) LOCAL. PRESTAÇÃO SERVIÇO", text)
    if match:
        local_prestacao_servico_value = match.group(1).strip()
        data_box_valores['issqn_retido'] = local_prestacao_servico_value 

    else:
        local_prestacao_servico_value = None
        data_box_valores['issqn_retido'] = local_prestacao_servico_value   
        
    match = re.search(r"LOCAL. PRESTAÇÃO SERVIÇO(.*?) LOCAL INCIDÊNCIA", text)
    if match:
        local_prestacao_servico_value = match.group(1).strip()
        data_box_valores['local_prestacao_servico'] = local_prestacao_servico_value

    else:
        local_prestacao_servico_value = None
        data_box_valores['local_prestacao_servico'] = local_prestacao_servico_value  
        
    match = re.search(r'LOCAL INCIDÊNCIA\s+(.+)', text)
    if match:
        local_incidencia_value = match.group(1).strip()
        data_box_valores['local_incidencia'] = local_incidencia_value
    else:
        local_incidencia_value = None
        data_box_valores['local_incidencia'] = local_incidencia_value   
        

    return data_box_valores

### <mark> <b>2.2</b> ExecuÇao do Pipeline de Extracao </mark>

In [ ]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, doc_content, fase, atividade, status, debug=False, prestador=True, tomador=True, servicos=True, total=True, cnae=True, valores_impostos=True, complementares=True, outras_informacoes=True, observacoes=True):
    
    doc_info = {}
    resumo = {}
    row_teste_info = []
    time_now = cron.timenow_pt_BR()
    func_fase = fase
    func_atividade = atividade
    func_status = status
    lista_dicts = []
    conf_processo = {}
    lista_conferencia = []
   
    i = 1
    for idx, row in qualquer_df.iterrows():
        dados_iniciais = {}
        row_info = row.to_dict()
        message_erro = []
        # 1. Mapeamento de informacoes do DF
        map_document_unique_id = idx
        map_seq = row['seq']
        map_batch_name = row['batch']
        map_fase_processo = row['fase_processo']
        map_nome_atividade = row['nome_atividade']
        map_status_documento = row['status_documento']
        map_original_file_name = row['original_file_name']
        map_directory = row['directory']
        map_one_page = row['one_page']
        map_palavra_chave = row['palavra_chave']
        map_document_tag = row['document_tag']
        map_action_item = row['action_item']
        map_pdf_pesquisavel = row['pdf_pesquisavel']
        map_level = row['level']
        file_path = row['file_path']
        row_info['document_unique_id'] = map_document_unique_id
    
        # XXX Nivel 1 - Definindo que documentos serao tratados   
        if map_status_documento == 'PREPROCESS_EXTRACT':
            print(f'seq: {i:>3} | {batch_name} | idx: {idx} | pdf pesq.: {map_pdf_pesquisavel} | status_documento: {map_status_documento} | processando doc: {map_original_file_name} ')
            # 1. Buscando o texto do documento pelo doc_content
            texto_dict = doc_content.get(map_document_unique_id, {}).get('content', 'valor_padrao')
            
            # 2. XXX IMPORTANTE - Efetuo a busca de entidades e efetuo a tokenizaÇao do documento
            doc, tokens, ents = show_ent_new(texto_dict, patterns=patterns)
            matches = matcher(doc)
                    
            prefeitura_map = [ent.orth_ for ent in doc.ents if ent.label_ == "nome_prefeitura"][0]
            de_para_pm = [ent.id_ for ent in doc.ents if ent.label_ == "nome_prefeitura"][0]
            secretaria_map = [ent.orth_ for ent in doc.ents if ent.label_ == "secretaria"][0]
            tipo_documento_map = [ent.orth_ for ent in doc.ents if ent.label_ == "tipo_documento"][0] 
         
            f_type = 'document'
            result = utl.filtrar_df(frames_nf_v4_df, type=f_type, de_para_pm=de_para_pm)
            model_map = result['model'].values[0]
            if debug:
                print(f'\nresult: {result}\n')
            
            section = "1. CABECALHO"
            valores = {}
            mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
            context_mapping = "data_cabecalho"
            def_replace = True
            
            if map_pdf_pesquisavel == False:
                imagem_gray, image_resized_name = convert_resize_gray(map_original_file_name, file_path, image_resized_path)
                imagem_gray_rgb = imagem_gray.convert("RGB")
                imagem_gray_np = np.array(imagem_gray_rgb)
                doc_content[map_document_unique_id]['image_np'] = imagem_gray_np

                # vou tratar o texto do cabecalho
                texto_cabecalho_PDF_Raster = extrac_cabecalho_R_PDF(idx, row, row_info, doc_content, section, mapping_method, context_mapping, map_pdf_pesquisavel, model_map, map_original_file_name, file_path, debug)
                #print(f'\n1. texto_cabechalho_PDF_Raster: {texto_cabechalho_PDF_Raster}\n')
                # Ajusto o texto todo
                if texto_cabecalho_PDF_Raster:
                    texto_recomposto = ajusta_texto_Raster_P(idx, row, map_document_unique_id, texto_dict, texto_cabecalho_PDF_Raster)
                    if debug:
                        print(f'\n2. texto_dict: {texto_recomposto}\n')
                    # Atualizao do doc_content
                    doc_content[map_document_unique_id]['content'] = texto_recomposto
                    texto_dict = texto_recomposto    
            
            #matcher = Matcher(nlp.vocab)
            matches = matcher(doc)
            
            # XXX Rotina para carregar e atribuir ao matcher os patterns do disco
            matcher_pattern_path = tipo_documento_dict.get(tipo_doc_work, {}).get('matcher_pattern_path', 'valor_padrao')
            # XXX Carregar matcher patterns do disco
            with open(matcher_pattern_path, "r") as f:
                loaded_patterns = json.load(f)
                
            # Adicionar ao Matcher
            for label, pattern in loaded_patterns.items():
                matcher.add(label, [pattern])  

            doc, tokens, ents = show_ent_new(texto_dict, patterns=patterns)
            matches = matcher(doc)
            
            # 3. XXX Mapeamento dados do cabecalho
            valores = mapeia_cabecalho(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, map_pdf_pesquisavel, model_map, map_original_file_name, file_path, debug)
            
            if debug:
                print(f'\nvalores: {valores}\n')
            
            row_info['model'] = model_map
            row_info['de_para_pm'] = de_para_pm
            row_info['tipo_nota_fiscal'] = tipo_documento_map
            row_info['secretaria'] = secretaria_map
            row_info['prefeitura'] = prefeitura_map
            if debug:
                print(f'1. row_info: {row_info}\n')
            
            row_info.update(valores)
            information_row_info = "Este e apenas um comeco - mas bem comeco mesmo"
            action_item_row_info = 'CONTINUE_PROCESS'

            # 4. XXX Mapeamento dados do prestador
            valores = {}
            valores = mapeia_prestador(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, map_pdf_pesquisavel, model_map, map_original_file_name, file_path, debug)
            if valores:
                row_info.update(valores)
                
            # 5. XXX Mapeamento dados do tomador    
            valores = {}    
            valores = mapeia_tomador(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, map_pdf_pesquisavel, model_map, map_original_file_name, file_path, debug)
            if valores:
                row_info.update(valores)
            
            # 4. DESCRIMINACAO DOS SERVIÇOS
            texto = doc_content.get(idx, {}).get('content', 'valor_padrao')
            servicos_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '4. DESCRIMINACAO DOS SERVIÇOS'][0]
            valor_total_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '5. VALOR TOTAL'][0]
            descr_servicos = texto[servicos_end_char:valor_total_start_char]
            row_info['discriminacao_servicos'] = descr_servicos     

            matches = matcher(doc)
            
            # XXX Rotina para carregar e atribuir ao matcher os patterns do disco
            matcher_pattern_path = tipo_documento_dict.get(tipo_doc_work, {}).get('matcher_pattern_path', 'valor_padrao')

            # XXX Carregar matcher patterns do disco
            with open(matcher_pattern_path, "r") as f:
                loaded_patterns = json.load(f)
                
            # Adicionar ao Matcher
            for label, pattern in loaded_patterns.items():
                matcher.add(label, [pattern])  

            doc, tokens, ents = show_ent_new(texto_dict, patterns=patterns)
            matches = matcher(doc)
            
            # 6. Mapeia cnae e Itens
            valores = {}    
            valores = mapeia_cnae_item(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, map_pdf_pesquisavel, model_map, map_original_file_name, file_path, debug)
            if valores:
                row_info.update(valores)
            
            
            # 7. Mapeia valores
            valores = {}                     
            valores = mapeia_campos_valores(idx, row, row_info, doc_content, doc, matches, section, map_pdf_pesquisavel, model_map, map_original_file_name, file_path, debug)
            if valores:
                row_info.update(valores)
            
            
            # 8. DADOS COMPLEMENTARES
            dados_complementares_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '8. DADOS COMPLEMENTARES'][0]
            outras_informacoes_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '9. OUTRAS INFORMAÇOES / CRITICAS'][0]
            dados_complementares = texto[dados_complementares_end_char:outras_informacoes_start_char]
            row_info['dados_complementares'] = dados_complementares
            
           # 9. Mapeia Informaçoes criticas
            valores = {}    
            valores = mapeia_informacoes_criticas(idx, row, row_info, doc_content, doc, matches, section, mapping_method, context_mapping, map_pdf_pesquisavel, model_map, map_original_file_name, file_path, debug)
            if valores:
                row_info.update(valores)     
                
            # 9. OUTRAS INFORMAÇOES / CRITICAS
            outras_informacoes_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '9. OUTRAS INFORMAÇOES / CRITICAS'][0]
            observacoes_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '10. OBSERVACOES'][0]
            observacao = texto[outras_informacoes_end_char:observacoes_start_char]
            row_info['observacao'] = observacao
            
            
            lista_dicts.append(row_info)
            i += 1
            # continue

        
        elif map_status_documento == 'NO_PROCESS' or map_status_documento == 'root_analise':
            msg = (f'Documento nao sera tratado neste escopo: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory}')
            row_info['action_item'] = "NO_PROCESS"    
            row_info['informations'] = msg 
            lista_dicts.append(row_info)
            #print(f'\nprocessando: batch: {batch_name} | seq: {i} | file_name: {map_original_file_name:>40} | idx: {idx} - Nao sera processado')
            i += 1
            continue

    #logging.info(f'processamento finalizado para: {batch_name}') 
    print(f'\n\nprocessamento de {i - 1} documentos')
    novo_df = pd.DataFrame(lista_dicts)
  
    return novo_df

In [ ]:
# analisar_pdf_pesquisavel
fase = 'analise'
atividade = 'PREPROCESS' 
status = 'PREPROCESS_EXTRACT'
lista_dicts = []
#logging.info(f'Execuçao do pipeline para {batch_name} | df_root_pipe: {file_path_root_pipe} fase: {fase} atividade: {atividade} status: {status}  template: {ver}')
# 1. Processar somente dados iniciais e cabeçalho
df_result_pipe = extracao_pipeline(df_root_pipe, doc_content, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

In [ ]:
# 2. XXX Volto novamente o indice do DF IMPORTANTE
df_result_pipe.set_index('document_unique_id', inplace=True)

ordem_status = ['PREPROCESS_EXTRACT', 'NO_PROCESS', 'root_analise']
ordem_action_item = ['CONTINUE_PROCESS', 'BREAK_PROCESS', 'NO_PROCESS']

df_result_pipe['status_documento'] = pd.Categorical(df['status_documento'], categories=ordem_status, ordered=True)
df_result_pipe['action_item'] = pd.Categorical(df['action_item'], categories=ordem_action_item, ordered=True)

df_result_pipe.sort_values(by=['status_documento', 'action_item', 'seq'], ascending=[True, True, True], inplace=True)

In [ ]:
df_result_pipe.columns

### <mark> <b>2.3</b> Conferencia Extracao </mark>

In [ ]:
# 1. Dados basicos do documento
df_conf0 = df_result_pipe[['batch', 'sigla_tipo', 'fase_processo', 'nome_atividade', 'nome_atividade', 'acao_executada', 'original_file_name', 'directory', 'pdf_pesquisavel', 'score', 'pages', 'document_tag', 'action_item']]
df_conf0

In [ ]:
# 2. Conferencia 1
df_conf1 = df_result_pipe[['original_file_name', 'directory', 'pdf_pesquisavel','model','tipo_nota_fiscal','secretaria','prefeitura', 'numero_nota_fiscal', 'competencia', 'dt_hr_emissao','codigo_verificacao']]
df_conf1

In [ ]:
# 3. Conferencia 2
df_conf2 = df_result_pipe[['original_file_name', 'directory', 'pdf_pesquisavel','p_cpf_cnpj_com_mascara', 'p_cpf_cnpj_sem_mascara', 'p_telefone', 'p_inscricao_municipal', 'p_inscricao_estadual', 'razao_social_prestador', 'p_nome_fantasia', 'endereco_prestador', 'p_email']]
df_conf2

In [ ]:
# 4. Conferencia 3
df_conf3 = df_result_pipe[['original_file_name', 'directory', 'pdf_pesquisavel','t_cpf_cnpj_com_mascara', 't_cpf_cnpj_sem_mascara', 't_inscricao_municipal', 't_telefone', 't_RG', 't_inscricao_estadual', 't_nome_razao_social', 't_endereco', 't_email']]
df_conf3

In [ ]:
# 5. Conferencia 4
df_conf4 = df_result_pipe[['original_file_name', 'directory', 'pdf_pesquisavel','discriminacao_servicos', 'valor_total_nota', 'cnae','item_lista_servicos']]
df_conf4

In [ ]:
# 6. Conferencia 5
df_conf5 = df_result_pipe[['original_file_name', 'directory', 'pdf_pesquisavel','valor_servicos', 'valor_deducao', 'base_calculo','aliquota', 'valor_iss', 'valor_iss_retido', 'desc_cond', 'valor_pis', 'valor_cofins', 'valor_ir', 'valor_inss', 'valor_csll','outras_retencoes', 'valor_liquido']]
df_conf5

In [ ]:
# 7. Conferencia 6
df_conf6 = df_result_pipe[['original_file_name', 'directory', 'pdf_pesquisavel','dados_complementares','exigibilidade_iss', 'regime_tributacao', 'simples_nacional', 'issqn_retido', 'local_prestacao_servico', 'local_incidencia', 'observacao']]
df_conf6

In [ ]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_df_result_pipe = os.path.join(map_analise_path, 'df_result_pipe_' + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_result_pipe
df_result_pipe.to_excel(file_path_df_result_pipe, index=True)

### <mark> <b>2.4</b> Exportacao json </mark>

In [ ]:
# 1 XXX. Leio a planilha de conferencia avalidada
file_path_df_result_pipe = os.path.join(map_analise_path, 'df_result_pipe_' + batch_name + ".xlsx")


#Le a planilha e cria do DF
df_validated_pipe = pd.read_excel(file_path_df_result_pipe)

In [ ]:
# 2. Crio subset somente dos documentos para exportar
df_conf_validada = utl.filtrar_df(df_validated_pipe, action_item="EXPORT")

In [ ]:
# 3. Ajusto o Index
df_conf_validada.set_index('document_unique_id', inplace=True)

In [ ]:
# 4. Dados para formataÇao do Json
de_para_pm = df_conf_validada['de_para_pm'].values[0]
municipio = df_conf_validada['municipio'].values[0]
arquivo_zip = df_conf_validada['parent_file'].values[0]
data_processamento = cron.timenow_pt_BR()
nome_formado_json = batch_name +".json"
titulo = (f'Processamento {batch_name} - {de_para_pm} - data:{data_processamento}')

In [ ]:
# 5. Crio o JSON 
dados_json = {}

# Iterar sobre cada linha no DataFrame
for index, row in df_conf_validada.iterrows():
    # dados_df e o dicionario para armazenar os dados da nota fiscal atual
    #diretorio = str(row['directory'])
    dados_nf = {
            "dados_NF_PDF": {
                                "data_cabecalho": {
                                    "secao": "1 - CABECALHO",
                                    "nome_prefeitura": row['prefeitura'],
                                    "numero_nota_fiscal": row['numero_nota_fiscal'],
                                    "competencia": row['competencia'],
                                    "dt_hr_emissoa": row['dt_hr_emissao'],
                                    "codigo_verificacao": row['codigo_verificacao']
                                },
                                "data_prestador": {
                                    "secao": "2. PRESTADOR DE SERVIÇO",
                                    "cpf_cnpj_com_mascara": row['p_cpf_cnpj_com_mascara'],
                                    "cpf_cnpj_sem_mascara": row['p_cpf_cnpj_sem_mascara'],
                                    "inscricao_municipal": row['p_inscricao_municipal'],
                                    "inscricao_estadual": row['p_inscricao_estadual'],
                                    "telefone": row['p_telefone'],
                                    "razao_social": row['razao_social_prestador'],
                                    "nome_fantasia": row['p_nome_fantasia'],
                                    "endereco": row['endereco_prestador'],
                                    "email": row['p_email']
                                },
                                "data_tomador": {
                                    "secao": "3. TOMADOR DE SERVIÇO",
                                    "cpf_cnpj_com_mascara": row['t_cpf_cnpj_com_mascara'],
                                    "cpf_cnpj_sem_mascara": row['t_cpf_cnpj_sem_mascara'],
                                    "rg": row['t_RG'],
                                    "inscricao_municipal": row['t_inscricao_municipal'],
                                    "inscricao_estadual": row['t_inscricao_estadual'],
                                    "telefone": row['t_telefone'],
                                    "razao_social": row['t_nome_razao_social'],
                                    "endereco": row['t_endereco'],
                                    "email": row['t_email']
                                },
                                "data_servico": {
                                    "secao": "4. DESCRIMINACAO DOS SERVIÇOS",
                                    "discriminacao_servicos": row['discriminacao_servicos']
                                },
                                "data_valor_total": {
                                    "secao": "5. VALOR TOTAL",
                                    "valor_total_nota": row['valor_total_nota']
                                },
                                "data_CNAE": {
                                    "secao": "6. CNAE e Item da Lista de Serviços",
                                    "cnae": row['cnae'],
                                    "item_lista_servicos": row['item_lista_servicos']
                                },
                                "data_valores": {
                                    "secao": "7. VALORES E IMPOSTOS",
                                    "valor_servicos": row['valor_servicos'],
                                    "valor_deducao": row['valor_deducao'],
                                    "desc_incond" : row['desc_incond'],
                                    "base_calculo": row['base_calculo'],
                                    "aliquota": row['aliquota'],
                                    "valor_iss": row['valor_iss'],
                                    "valor_iss_retido": row['valor_iss_retido'],
                                    "desc_cond": row['desc_cond'],
                                    "valor_pis": row['valor_pis'],
                                    "valor_cofins": row['valor_cofins'],
                                    "valor_ir": row['valor_ir'],
                                    "valor_inss": row['valor_inss'],
                                    "valor_csll": row['valor_csll'],
                                    "outras_retencoes": row['outras_retencoes'],
                                    "valor_liquido": row['valor_liquido']
                                },
                                "data_dados_complementares": {
                                    "secao": "8. DADOS COMPLEMENTARES",
                                    "dados_complementares": row['dados_complementares']
                                },
                                "data_outras_informacoes": {
                                    "secao": "9. OUTRAS INFORMAÇOES / CRITICAS",
                                    "exigibilidade_iss": row['exigibilidade_iss'],
                                    "regime_tributacao": row['regime_tributacao'],
                                    "simples_nacional": row['simples_nacional'],
                                    "issqn_retido": row['issqn_retido'],
                                    "local_prestacao_servico": row['local_prestacao_servico'],
                                    "local_incidencia": row['local_incidencia']
                                },
                                "data_observacao": {
                                    "secao": "10. OBSERVACOES",
                                    "observacao": row['observacao']
                                },
                            },
                            "batch": row['batch'],    
                            "diretorio": str(row['directory']),
                            "nome_arquivo": row['original_file_name'],
                            "pdf_pesquisavel": row['pdf_pesquisavel'],
                            "modelo": row['model'],   
                            "document_unique_id": index,
                            "parent_file": row['parent_file'],
                    }        
            
    
    numero_nota_fiscal = str(row['numero_nota_fiscal'])
    dados_json['titulo'] = titulo
    dados_json['batch'] = batch_name
    dados_json['municipio'] = municipio
    # dados_json['data_processamento'] = cron.timenow_pt_BR()
    dados_json[numero_nota_fiscal] = dados_nf

# Salvando em formato JSON
json_file_path = os.path.join(json_path, nome_formado_json)
with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(dados_json, f, ensure_ascii=False, indent=4)
    
print(f"As informações foram salvas em {json_file_path}")  

In [ ]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_df_conf_export = os.path.join(map_analise_path, 'df_conf_export_' + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_result_pipe
df_validated_pipe.to_excel(file_path_df_conf_export, index=True)

## Dev container

In [ ]:
df_conf_validada['de_para_pm'].values[0]
df_conf_validada['municipio'].values[0]
df_conf_validada['parent_file'].values[0]

In [ ]:
df['coluna_cnae'] = df['coluna_cnae'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
df_conf_avalida_batch_21['numero_nota_fiscal'] = df_conf_avalida_batch_21['numero_nota_fiscal'].astype(int)

### Ajustes Matchs

In [ ]:
# XXX 1.Processar todas as secoes do documento
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=True, tomador=True, servicos=True, total=True, cnae=True, valores_impostos=True, complementares=True, outras_informacoes=True, observacoes=True)



# 5. Processar valor Total
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=True, cnae=False, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

# 6. Processar CNAE
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=True, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

# 7. Processar Impostos
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=, complementares=False, outras_informacoes=False, observacoes=False)

# 8. complementar e observaçoes
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=False, complementares=True, outras_informacoes=True, observacoes=True)

In [ ]:
matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f"{string_id:>30} | {span.text:>50} | {span.start_char:>5}  {span.end_char:>5}")

In [ ]:
raw_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == label), None)



In [ ]:
most_similar_reference = max([reference], key=lambda x: similar(x, raw_value))

In [ ]:
df_root_pipe.head()

In [ ]:
document_unique_id = "ab2457b7-ea5c-4191-acf0-bc8edc04879e"
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_23/MESQUITA_PDF_31282023_2258/teste/1.pdf"

In [ ]:
# 4. DESCRIMINACAO DOS SERVIÇOS
servicos_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '4. DESCRIMINACAO DOS SERVIÇOS'][0]
valor_total_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '5. VALOR TOTAL'][0]
bloco_discriminacao_servico = texto[servicos_end_char:valor_total_start_char]
bloco_discriminacao_servico

In [ ]:
document_unique_id = 'ab2457b7-ea5c-4191-acf0-bc8edc04879e'
texto = doc_content.get(document_unique_id, {}).get('content', 'valor_padrao')
doc = nlp(texto)
texto

In [ ]:
# 9. OUTRAS INFORMAÇOES / CRITICAS
outras_informacoes_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '9. OUTRAS INFORMAÇOES / CRITICAS'][0]
observacoes_start_char = [ent.start_char for ent in doc.ents if ent.id_ == '10. OBSERVACOES'][0]
text = texto[outras_informacoes_end_char:observacoes_start_char]
text

In [ ]:
# 9. OUTRAS INFORMAÇOES / CRITICAS
def extract_fields_outras_info(text):
    nf_data_outras_informacoes = {}
    #nf_data_outras_informacoes['secao'] = "9. OUTRAS INFORMAÇOES / CRITICAS"
    
    # Extrair EXIGIBILIDADE ISS:
    exigibilidade_iss_match = re.search(r'EXIGIBILIDADE ISS\s+(.+)', text)
    if exigibilidade_iss_match:
        exigibilidade_iss_value = exigibilidade_iss_match.group(1).strip()
        nf_data_outras_informacoes['exigibilidade_iss'] = exigibilidade_iss_value
        
    # Extrair REGIME TRIBUTAÇÃO:
    regime_tributacao_match = re.search(r'REGIME TRIBUTAÇÃO\s+(.+)', text)
    if regime_tributacao_match:
        regime_tributacao_value = regime_tributacao_match.group(1).strip()
        nf_data_outras_informacoes['regime_tributacao'] = regime_tributacao_value
    
    # Extrair SIMPLES NACIONAL:
    simples_nacional_match = re.search(r'SIMPLES NACIONAL\s+(.+)', text)
    if simples_nacional_match:
        simples_nacional_value = simples_nacional_match.group(1).strip()
        nf_data_outras_informacoes['simples_nacional'] = simples_nacional_value
        
        
    # Extrair ISSQN RETIDO:
    local_prestacao_servico_match = re.search(r'ISSQN RETIDO\s+(.+)', text)
    if local_prestacao_servico_match:
        local_prestacao_servico_value = local_prestacao_servico_match.group(1).strip()
        nf_data_outras_informacoes['issqn_retido'] = local_prestacao_servico_value        
        
    
    # Extrair LOCAL PRESTAÇÃO SERVIÇO:
    local_prestacao_servico_match = re.search(r'LOCAL\. PRESTAÇÃO\s+SERVIÇO\s+(.+)', text)
    if local_prestacao_servico_match:
        local_prestacao_servico_value = local_prestacao_servico_match.group(1).strip()
        nf_data_outras_informacoes['local_prestacao_servico'] = local_prestacao_servico_value
    
    
    
    # Extrair LOCAL INCIDÊNCIA:
    local_incidencia_match = re.search(r'LOCAL INCIDÊNCIA\s+(.+)', text)
    if local_incidencia_match:
        local_incidencia_value = local_incidencia_match.group(1).strip()
        nf_data_outras_informacoes['local_incidencia'] = local_incidencia_value

In [ ]:
outras_ref = ['EXIGIBILIDADE ISS', 'REGIME TRIBUTAÇÃO', 'SIMPLES NACIONAL', 'ISSQN RETIDO', 'LOCAL. PRESTAÇÃO SERVIÇO', 'LOCAL INCIDÊNCIA']

In [ ]:
novo_texto = "EXIGIBILIDADE ISS\nExigivel\nREGIME TRIBUTAÇÃO\nSociedade Limitada\nSIMPLES NACIONAL\nSim (2,01% )\nISSQN RETIDO\nNão\nLOCAL. PRESTAÇÃO SERVIÇO\nMagé - RJ\nLOCAL INCIDÊNCIA\nMagé - RJ" 

In [ ]:
data_outras_informacoes = {}
data_outras_informacoes = novaextra.extract_fields_outras_info(novo_texto)
data_outras_informacoes

In [ ]:
data_outras_informacoes

In [ ]:
# 10. OBSERVACOES
observacoes_end_char = [ent.end_char for ent in doc.ents if ent.id_ == '10. OBSERVACOES'][0]
encerrador_start_char = [ent.start_char for ent in doc.ents if ent.label_ == 'encerrador'][0]
bloco_observacoes = texto[observacoes_end_char:encerrador_start_char]
bloco_observacoes


In [ ]:
# Analisys
syntatic = pd.DataFrame(data=[], \
  columns=["id", "T_texto","T_shape", "T_is_alpha", "T_is_digit", "T_is_title", "T_is_punct", "T_is_sent_start", "T_is_right_punct", "T_is_stop", "T_is_quote", "T_is_currency", "T_morph"])
i = 0
for token in doc:
    syntatic.loc[i,"id"] = token.i
    syntatic.loc[i,"T_texto"] = token.text
    syntatic.loc[i,"T_shape"] = token.shape_
    syntatic.loc[i,"T_is_alpha"] = token.is_alpha
    syntatic.loc[i,"T_is_digit"] = token.is_digit
    syntatic.loc[i,"T_is_title"] = token.is_title
    syntatic.loc[i,"T_is_punct"] = token.is_punct
    syntatic.loc[i,"T_is_sent_start"] = token.is_sent_start
    syntatic.loc[i,"T_is_right_punct"] = token.is_right_punct
    syntatic.loc[i,"T_is_stop"] = token.is_stop
    syntatic.loc[i,"T_is_quote"] = token.is_quote
    syntatic.loc[i,"T_is_currency"] = token.is_currency
    syntatic.loc[i,"T_morph"] = token.morph
    i = i+1

syntatic.head(80)

In [ ]:
# Posições de início e fim
start_pos = 881
end_pos = 1520

# Extraindo a sub-string usando slicing
extracted_text = texto[start_pos:end_pos]

print(f"Texto extraído: {extracted_text}")


In [ ]:
bloco_discriminacao_servico = original_text[servicos_end_char:valor_total_start_char]
bloco_discriminacao_servico

In [ ]:
servicos_end = [ent.end for ent in doc.ents if ent.id_ == '4. DESCRIMINACAO DOS SERVIÇOS'][0]
servicos_end

In [ ]:
valor_total_start = [ent.start for ent in doc.ents if ent.id_ == '5. VALOR TOTAL'][0]
valor_total_start

In [ ]:
bloco_discriminacao_servico = texto[servicos_end:valor_total_start]
bloco_discriminacao_servico

### Parte do mapeamento do cabeÇalho para Raster_PDF

In [ ]:
document_unique_id = 'ab2457b7-ea5c-4191-acf0-bc8edc04879e'
texto_R_PDF = doc_content.get(document_unique_id, {}).get('content', 'valor_padrao')
texto_R_PDF

In [ ]:
texto_R_PDF = Novo_texto

In [ ]:
doc, tokens, ents = show_ent_new(texto_R_PDF, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

In [ ]:
valores = {}
section = "2. PRESTADOR DE SERVIÇO"
mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
context_mapping = "data_prestador"
model_map = "MESQUITA"
debug = False

In [ ]:
# Buscando a razao social - Prestador
label = "p_razao_social"
end_chars_razao_prestado = [token.idx + len(token) for token in doc if token.ent_id_ == label][4]
end_chars_razao_prestado = end_chars_razao_prestado + 1
label = "p_nome_fantasia"
star_chars_fantasia = [token.idx for token in doc if token.ent_id_ == label][0]
star_chars_fantasia = star_chars_fantasia - 1
razao_social_texto = texto_R_PDF[end_chars_razao_prestado:star_chars_fantasia]
razao_social_texto

In [ ]:

ini_char = min(end_chars)
end_char = max(end_chars)

fatiado = doc.text[ini_char:end_char]

print(f'ini_char: {ini_char} | end_char: {end_char} | fatiado: {fatiado} \n')

In [ ]:
data_box_valores = {}
if mapping_method == "frame_&_sframe_field":
    tipo_4_coordinates = "frame"
    tipo_4_filter = "sframe_field"

# 8. Efetuo o filtro para a iteracao
filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]

# 9. iter sobre o filtro
for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
    section = row_frame['section_json']
    label = row_frame['label']
    reference = row_frame['reference']
    string_pesquisa = row_frame['marcador_inicio'] 
    # last_token_end = 0
    if label == "p_inscricao_municipal":
        last_token_end = 48
        new_token_start = 160
        print(f'label: {label}\n')
        
        end_chars = [token.idx + len(token) for token in doc if token.ent_id_ == label]
        ini_char = min(end_chars)
        end_char = max(end_chars)
        
        fatiado = doc.text[ini_char:end_char]
        
        print(f'ini_char: {ini_char} | end_char: {end_char} | fatiado: {fatiado} \n')
        
        syntatic = pd.DataFrame(data=[], \
        columns=["id", "T_texto", "T_idx", "T_start_char", "T_end_char", "T_shape", "T_ent_id_", "T_ent_type_"])
        
        i = 0
        for token in doc[last_token_end:new_token_start]:
            
            start_char = token.idx
            end_char = start_char + len(token)
            syntatic.loc[i,"id"] = token.i
            syntatic.loc[i,"T_texto"] = token.text
            syntatic.loc[i,"T_idx"] = token.idx
            syntatic.loc[i,"T_start_char"] = start_char 
            syntatic.loc[i,"T_end_char"] = end_char
            syntatic.loc[i,"T_shape"] = token.shape_
            syntatic.loc[i,"T_ent_id_"] = token.ent_id_
            syntatic.loc[i,"T_ent_type_"] = token.ent_type_
            token_end_char = end_char


            i = i+1
        last_token_end = ent.end_char    

syntatic.head(50)

In [ ]:
section = "1. CABECALHO"
valores = {}
mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
context_mapping = "data_cabecalho"
def_replace = True
model_map = "MAGE"
debug = False

In [ ]:
label = 'numero_nota'

In [ ]:
raw_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == label), None)
raw_value

### Recomposicao do documento

In [ ]:
data_box_valores

In [ ]:
texto_cabechalho_PDF_Raster

In [ ]:
doc_c = nlp(texto_cabechalho_PDF_Raster)

In [ ]:
document_unique_id = '27df9e70-b5fb-45b7-8f59-0c04ed9728e2'
texto_R_PDF = doc_content.get(document_unique_id, {}).get('content', 'valor_padrao')
texto_R_PDF

In [ ]:
document_unique_id = 'ab2457b7-ea5c-4191-acf0-bc8edc04879e'
texto_PDF_P = doc_content.get(document_unique_id, {}).get('content', 'valor_padrao')
texto_PDF_P

In [ ]:
doc, tokens, ents = show_ent_new(texto_PDF_P, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

In [ ]:
matches = matcher(doc_c)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f"{string_id}: {span.text}")

In [ ]:
for ent in doc.ents:
    print(f'{ent.start:>5} | {ent.text:>50} | {ent.label_:>25} | {ent.id_:>35}  |   {ent.end:>4}   ||   {ent.start_char:>6} | {ent.end_char:>6}')

In [ ]:
# Seu texto OCR completo
original_text = texto_R_PDF
original_text

In [ ]:
doc, tokens, ents = show_ent_new(original_text, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

In [ ]:
doc, tokens, ents = show_ent_new(recomposed_text, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

In [ ]:
for ent in doc.ents:
    print(f'{ent.start:>5} | {ent.text:>50} | {ent.label_:>25} | {ent.id_:>35}  |   {ent.end:>4}   ||   {ent.start_char:>6} | {ent.end_char:>6}')

In [ ]:
#texto_amostra = "NFS-e Número da Nota: 20234 Competência: Julho/2023"
texto_amostra = "'outras informações / criticas'"

doc = nlp(texto_amostra)
matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f"{string_id}: {span.text}")

In [ ]:
# Analisys
syntatic = pd.DataFrame(data=[], \
  columns=["id", "T_texto","T_shape", "T_is_alpha", "T_is_digit", "T_is_title", "T_is_punct", "T_is_sent_start", "T_is_right_punct", "T_is_stop", "T_is_quote", "T_is_currency", "T_morph"])
i = 0
for token in doc:
    syntatic.loc[i,"id"] = token.i
    syntatic.loc[i,"T_texto"] = token.text
    syntatic.loc[i,"T_shape"] = token.shape_
    syntatic.loc[i,"T_is_alpha"] = token.is_alpha
    syntatic.loc[i,"T_is_digit"] = token.is_digit
    syntatic.loc[i,"T_is_title"] = token.is_title
    syntatic.loc[i,"T_is_punct"] = token.is_punct
    syntatic.loc[i,"T_is_sent_start"] = token.is_sent_start
    syntatic.loc[i,"T_is_right_punct"] = token.is_right_punct
    syntatic.loc[i,"T_is_stop"] = token.is_stop
    syntatic.loc[i,"T_is_quote"] = token.is_quote
    syntatic.loc[i,"T_is_currency"] = token.is_currency
    syntatic.loc[i,"T_morph"] = token.morph
    i = i+1

syntatic.head(80)

In [ ]:
clean_text = re.sub('\s+', ' ', texto_OCR_R).strip()

clean_text = text.replace(': ', ':').replace(', ', ',')

clean_text = re.sub('\s+', ' ', text.replace(': ', ':').replace(', ', ',')).strip()

text_splited = texto.split('\n')
text_splited = [x for x in text_splited if x.strip()]
text_splited = [s.replace(";", "").strip() for s in text_splited]